# Storing Financial Data Efficiently

In [105]:
import pandas as pd
from helpers.sample_generate import generate_sample_data

In [106]:
print(generate_sample_data(rows=5, cols=4, freq='1min'))

                            No0         No1         No2         No3
2023-01-01 00:00:00  100.000000  100.000000  100.000000  100.000000
2023-01-01 00:01:00  100.020396  100.029745  100.118653   99.998519
2023-01-01 00:02:00  100.078400  100.107048  100.063882   99.937550
2023-01-01 00:03:00  100.188669  100.260100  100.037083   99.998358
2023-01-01 00:04:00  100.327860  100.161058  100.165164   99.949647


## HDFStore

### .h5 for immutable data

In [107]:
%time data = generate_sample_data(rows=1e3, cols=10, freq='D').round(4)

Wall time: 8.94 ms


In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2023-01-01 to 2025-09-26
Freq: D
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No0     1000 non-null   float64
 1   No1     1000 non-null   float64
 2   No2     1000 non-null   float64
 3   No3     1000 non-null   float64
 4   No4     1000 non-null   float64
 5   No5     1000 non-null   float64
 6   No6     1000 non-null   float64
 7   No7     1000 non-null   float64
 8   No8     1000 non-null   float64
 9   No9     1000 non-null   float64
dtypes: float64(10)
memory usage: 85.9 KB


### HDFStore write to .h5

In [109]:
h5 = pd.HDFStore('data/data.h5', 'w')

In [110]:
%time h5['data'] = data

Wall time: 10.2 ms


In [111]:
h5

<class 'pandas.io.pytables.HDFStore'>
File path: data/data.h5

In [112]:
h5.close()

In [113]:
ls -n data

 Volume in drive C is OS
 Volume Serial Number is 8A40-DA6B

 Directory of C:\Users\Michael\source\algo-trading-examples


 Directory of C:\Users\Michael\source\algo-trading-examples\data

2023/12/25  23:02    <DIR>          .
2023/12/25  23:02    <DIR>          ..
2023/12/27  10:49           176,760 aapl.json
2023/12/27  10:49           129,024 aapl.xls
2023/12/27  10:49            58,640 aapl.xlsx
2023/12/13  22:41        29,580,549 all_stocks_5yr.csv
2023/12/27  10:50            95,240 data.h5
2023/12/25  18:27           152,942 data2.h5
2023/12/25  23:01           245,030 pyalgo_eikon_eod_data.csv
               7 File(s)     30,438,185 bytes
               2 Dir(s)  62,498,336,768 bytes free


File Not Found


### HDFStore read from .h5

In [114]:
h5 = pd.HDFStore('data/data.h5', 'r')
%time data_copy = h5['data']

Wall time: 6.99 ms


In [115]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2023-01-01 to 2025-09-26
Freq: D
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No0     1000 non-null   float64
 1   No1     1000 non-null   float64
 2   No2     1000 non-null   float64
 3   No3     1000 non-null   float64
 4   No4     1000 non-null   float64
 5   No5     1000 non-null   float64
 6   No6     1000 non-null   float64
 7   No7     1000 non-null   float64
 8   No8     1000 non-null   float64
 9   No9     1000 non-null   float64
dtypes: float64(10)
memory usage: 85.9 KB


In [116]:
h5.close()

### Pandas write to .h5 with to_hdf()

In [117]:
data.to_hdf('data/data2.h5', 'data', format='table')

In [118]:
data_copy2 = pd.read_hdf('data/data2.h5', 'data')

In [119]:
data_copy2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000 entries, 2023-01-01 to 2025-09-26
Freq: D
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No0     1000 non-null   float64
 1   No1     1000 non-null   float64
 2   No2     1000 non-null   float64
 3   No3     1000 non-null   float64
 4   No4     1000 non-null   float64
 5   No5     1000 non-null   float64
 6   No6     1000 non-null   float64
 7   No7     1000 non-null   float64
 8   No8     1000 non-null   float64
 9   No9     1000 non-null   float64
dtypes: float64(10)
memory usage: 85.9 KB


### Reading .h5 using PyTables (tables)

In [120]:
import tables as tb

In [121]:
h5 = tb.open_file('data/data.h5.', 'r')

In [122]:
h5

File(filename=data/data.h5., title='', mode='r', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/data (Group) ''
/data/axis0 (Array(10,)) ''
  atom := StringAtom(itemsize=3, shape=(), dflt=b'')
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := None
/data/axis1 (Array(1000,)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None
/data/block0_items (Array(10,)) ''
  atom := StringAtom(itemsize=3, shape=(), dflt=b'')
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := None
/data/block0_values (Array(1000, 10)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [123]:
h5.close()

## TsTables .h5ts for mutable and time series data

In [124]:
%%time
data = generate_sample_data(rows=2.5e6, cols=5,
                            freq='1s').round(4)

Wall time: 1.2 s


In [125]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2500000 entries, 2023-01-01 00:00:00 to 2023-01-29 22:26:39
Freq: S
Data columns (total 5 columns):
 #   Column  Dtype  
---  ------  -----  
 0   No0     float64
 1   No1     float64
 2   No2     float64
 3   No3     float64
 4   No4     float64
dtypes: float64(5)
memory usage: 114.4 MB


In [126]:
# pip install git+https://github.com/yhilpisch/tstables.git
import tstables
import tables as tb

In [127]:
class desc(tb.IsDescription):
    '''Description of TsTables table structure
    '''
    timestamp = tb.Int64Col(pos=0)
    N0 = tb.Float64Col(pos=1)
    N1 = tb.Float64Col(pos=2)
    N2 = tb.Float64Col(pos=3)
    N3 = tb.Float64Col(pos=4)
    N4 = tb.Float64Col(pos=5)

In [128]:
h5ts = tb.open_file('data/data.h5ts', 'w')
ts = h5ts.create_ts('/', 'data', desc)

In [129]:
h5ts

File(filename=data/data.h5ts, title='', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/data (Group/Timeseries) ''
/data/y2023 (Group) ''
/data/y2023/m12 (Group) ''
/data/y2023/m12/d27 (Group) ''
/data/y2023/m12/d27/ts_data (Table(0,)) ''
  description := {
  "timestamp": Int64Col(shape=(), dflt=0, pos=0),
  "N0": Float64Col(shape=(), dflt=0.0, pos=1),
  "N1": Float64Col(shape=(), dflt=0.0, pos=2),
  "N2": Float64Col(shape=(), dflt=0.0, pos=3),
  "N3": Float64Col(shape=(), dflt=0.0, pos=4),
  "N4": Float64Col(shape=(), dflt=0.0, pos=5)}
  byteorder := 'little'
  chunkshape := (1365,)

In [130]:
%time ts.append(data)

Wall time: 443 ms


In [131]:
h5ts

File(filename=data/data.h5ts, title='', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/data (Group/Timeseries) ''
/data/y2023 (Group) ''
/data/y2023/m01 (Group) ''
/data/y2023/m12 (Group) ''
/data/y2023/m12/d27 (Group) ''
/data/y2023/m12/d27/ts_data (Table(0,)) ''
  description := {
  "timestamp": Int64Col(shape=(), dflt=0, pos=0),
  "N0": Float64Col(shape=(), dflt=0.0, pos=1),
  "N1": Float64Col(shape=(), dflt=0.0, pos=2),
  "N2": Float64Col(shape=(), dflt=0.0, pos=3),
  "N3": Float64Col(shape=(), dflt=0.0, pos=4),
  "N4": Float64Col(shape=(), dflt=0.0, pos=5)}
  byteorder := 'little'
  chunkshape := (1365,)
/data/y2023/m01/d01 (Group) ''
/data/y2023/m01/d01/ts_data (Table(86400,)) ''
  description := {
  "timestamp": Int64Col(shape=(), dflt=0, pos=0),
  "N0": Float64Col(shape=(), dflt=0.0, pos=1),
  "N1": Float64Col(shape=(), dflt=0.0, pos=2),
  "N2": Float64Col(shape=(), dflt=0.

### Read .h5ts into DataFrame

In [132]:
import datetime
start = datetime.datetime(2023, 1, 2)
end = datetime.datetime(2023, 1, 3)

In [133]:
%time subset = ts.read_range(start, end)

Wall time: 32.1 ms


In [134]:
start = datetime.datetime(2023, 1, 2, 12, 30, 0)
end = datetime.datetime(2023, 1, 5, 17, 15, 30)
%time subset2 = ts.read_range(start, end)

Wall time: 57.4 ms


In [135]:
subset2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 276331 entries, 2023-01-02 12:30:00 to 2023-01-05 17:15:30
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   N0      276331 non-null  float64
 1   N1      276331 non-null  float64
 2   N2      276331 non-null  float64
 3   N3      276331 non-null  float64
 4   N4      276331 non-null  float64
dtypes: float64(5)
memory usage: 12.6 MB


In [136]:
h5ts.close()

In [137]:
# In conda prompt, run "del data\data.h5ts" to remove file

## Storing Data with SQLite3

#### Note: SQLite3 is a relational database, where the SQL query language can be applied to implement more sophisticated analyses. But it is slower than the binary storage format like HDF5. 

In [138]:
%time data = generate_sample_data(1e6, 5, '1min').round(4)

Wall time: 486 ms


In [139]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1000000 entries, 2023-01-01 00:00:00 to 2024-11-25 10:39:00
Freq: T
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   No0     1000000 non-null  float64
 1   No1     1000000 non-null  float64
 2   No2     1000000 non-null  float64
 3   No3     1000000 non-null  float64
 4   No4     1000000 non-null  float64
dtypes: float64(5)
memory usage: 45.8 MB


In [140]:
import sqlite3 as sq3
con = sq3.connect('data/data.sql')

In [141]:
%time data.to_sql('data', con)

Wall time: 5.36 s


1000000

In [142]:
query = 'SELECT * FROM data WHERE No1 > 105 and No2 < 108'

In [143]:
%time res = con.execute(query).fetchall()

Wall time: 1.76 s


In [144]:
res[-20:]

[('2024-11-25 10:20:00', 39.1159, 144.3006, 28.4391, 163.0812, 84.3871),
 ('2024-11-25 10:21:00', 39.1052, 144.3274, 28.4243, 163.0772, 84.447),
 ('2024-11-25 10:22:00', 39.1296, 144.3108, 28.4422, 163.0047, 84.3761),
 ('2024-11-25 10:23:00', 39.0868, 144.3319, 28.4531, 162.9157, 84.355),
 ('2024-11-25 10:24:00', 39.0826, 144.5099, 28.4673, 162.9825, 84.2849),
 ('2024-11-25 10:25:00', 39.0865, 144.6434, 28.4559, 162.9365, 84.2503),
 ('2024-11-25 10:26:00', 39.0423, 144.4878, 28.4772, 163.0169, 84.229),
 ('2024-11-25 10:27:00', 39.0051, 144.6625, 28.499, 162.9223, 84.2602),
 ('2024-11-25 10:28:00', 39.0088, 144.67, 28.5323, 162.8128, 84.2443),
 ('2024-11-25 10:29:00', 39.0461, 144.6678, 28.5391, 162.6645, 84.1988),
 ('2024-11-25 10:30:00', 39.1107, 144.5276, 28.5197, 162.6903, 84.2264),
 ('2024-11-25 10:31:00', 39.0876, 144.655, 28.538, 162.5758, 84.3),
 ('2024-11-25 10:32:00', 39.095, 144.6257, 28.497, 162.5737, 84.2481),
 ('2024-11-25 10:33:00', 39.1498, 144.6761, 28.4742, 162.7299, 8

In [145]:
con.close()

In [146]:
# In conda prompt, run "del data\data.sql" to remove file

### See also: SQLAlchemy, which allows for the use of MySQL as the relational database backend.